In [2]:
##############
#K. David Roell CFPB 
#collects row-level and counts of data that fail distribution style quality edts
#requires precomputed bins loaded as bin_df from ../linked_dist_stats/IQR_all.csv
#this file is generated using the IQR cleaning viz script which uses data from the get_linked_dist script
##############
from collections import OrderedDict
import json
import os
import pandas as pd
import psycopg2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from math import sqrt
get_ipython().magic(u'matplotlib inline')
matplotlib.style.use('ggplot')

with open('../quality_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
    edit_sql = json.load(f)
    
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',
}

try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)
    
#read csv of distributions
bin_df = pd.read_csv('../linked_dist_stats/IQR_all.csv')
bin_df.drop('Unnamed: 0', inplace=True, axis=1)
bin_df.tail()

i'm connected


,name,std,mean,median,max_mult,med_mult,min_mult,q98,q02,q75,q25,iqr,min_q,max_q,count of loans,pct of total loans
328,9830 to 9860,764.961157,1049.714286,1000.0,0.224343,0.101709,0.015218,2160.80,165.00,1525.00,469.0,1056.00,-1115.000,3109.000,8,0.000347
329,9860 to 9890,381.837662,1620.000000,1620.0,0.191567,0.164138,0.136709,1879.20,1360.80,1755.00,1485.0,270.00,1080.000,2160.000,2,0.000087
330,9890 to 9920,1028.012484,1686.000000,1313.0,0.303153,0.132680,0.032255,2985.60,386.32,2550.00,1035.0,1515.00,-1237.500,4822.500,9,0.000390
331,9920 to 9950,1201.301066,2054.250000,2292.5,0.324264,0.230615,0.041692,3180.02,509.16,2743.25,1603.5,1139.75,-106.125,4452.875,4,0.000173
332,9950 to 9980,1205.352749,1520.666667,1000.0,0.376015,0.100432,0.025100,3599.76,262.00,2000.00,550.0,1450.00,-1625.000,4175.000,11,0.000477


In [3]:
#get counts based on new distribution method
def get_count_edit_fails(table=None, inc_max=None, inc_min=None, amt_min=None, amt_max=None, name=None, prop_type="'1', '2', '3'", loan_purp="'1','2','3'", loan_type="'1','2','3','4'", occupancy="'1','2','3'", lien="'1','2','3'"):
    """generates a SQL statement to get counts of loans outside of expected distributions"""
    
    #set base selection and table
    base_SQL = """SELECT '{name}' AS bin, COUNT(amount) FROM {table} """.format(table=table, name=name)
    
    #set conditions for edit failure
    where_SQL = """WHERE (amount::INT NOT BETWEEN {amt_min} AND {amt_max}) AND
    (income::INT BETWEEN {inc_min} AND {inc_max}) """.format(amt_min=amt_min, amt_max=amt_max, inc_min=inc_min, inc_max=inc_max)
    
    filter_SQL = """AND property_type IN ({prop_type}) AND loan_type IN ({loan_type}) AND 
    loan_purpose IN ({loan_purp}) AND occupancy IN ({occupancy}) AND lien IN ({lien}) 
    """.format(prop_type=prop_type, loan_type=loan_type, loan_purp=loan_purp, occupancy=occupancy, lien=lien)
    #remove NA values from numeric fields
    na_SQL = """AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;"""

    final_SQL = base_SQL + where_SQL + filter_SQL + na_SQL #combine SQL strings to form query

    return final_SQL
    #sum failed rows



In [4]:
#get rows of failed edits with bin label
def get_row_edit_fails(table=None, inc_max=None, inc_min=None, amt_min=None, amt_max=None, name=None, prop_type="'1', '2', '3'", loan_purp="'1','2','3'", loan_type="'1','2','3','4'", occupancy="'1','2','3'", lien="'1','2','3'"):
    """generates a SQL statement to get rows of loans outside of expected distributions"""
    
    #set base selection and table
    base_SQL = """SELECT '{name}' AS bin, agency, RID, income, amount, sequence FROM {table} """.format(table=table,  name=name)
    
    #set conditions for edit failure
    where_SQL = """WHERE (amount::INT NOT BETWEEN {amt_min} AND {amt_max}) AND
    (income::INT BETWEEN {inc_min} AND {inc_max}) """.format(amt_min=amt_min, amt_max=amt_max, inc_min=inc_min, inc_max=inc_max)
    
    filter_SQL = """AND property_type IN ({prop_type}) AND loan_type IN ({loan_type}) AND 
    loan_purpose IN ({loan_purp}) AND occupancy IN ({occupancy}) AND lien IN ({lien}) 
    """.format(prop_type=prop_type, loan_type=loan_type, loan_purp=loan_purp, occupancy=occupancy, lien=lien)
    #remove NA values from numeric fields
    na_SQL = """AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;"""

    final_SQL = base_SQL + where_SQL + filter_SQL + na_SQL #combine SQL strings to form query

    return final_SQL
    #sum failed rows



In [ ]:
#get edit fails based on distribution method
#this cell retrieves counts of edit fails by year (year must be adjusted manually)
#FIXME: add second query to get individual records that failed

#kwargs are used to set filters for market product
#this script should be tested with all loan types and property types
kwargs= {'table':'hmdalar2014',
         'prop_type':"'1'",
        'loan_purp':"'1'",
        'loan_type':"'1'",
        'occupancy':"'1'", 
        'lien':"'1'"}


first = True #set boolean flag to instantiate first instance of dataframe

for index, row in bin_df.iterrows(): #iterate over bins of income from bin_df 
#    print(index)
    #set variable arguments for sql query filters
    kwargs['name'] = bin_df.name.loc[index]
    kwargs['inc_min'] = int(kwargs['name'].rsplit(' ', 2)[0])
    kwargs['inc_max'] = int(kwargs['name'].rsplit(' ', 2)[2])
    kwargs['amt_min'] = bin_df.q02.loc[index]
    kwargs['amt_max'] = bin_df.q98.loc[index]
    
    sql = get_count_edit_fails(**kwargs) #format query using kwargs dictionary
    print(sql) #check query text
    cur.execute(sql,) #execute SQL
    data_df = pd.DataFrame(cur.fetchall()) #convert results to dataframe
    
    col_names = [desc[0] for desc in cur.description] # get column names from SQL table    
    if len(data_df.columns) == len(col_names) and first: #name dataframe columns if query not empty

        print('setting column names')
        data_df.columns = col_names

    if first == True and len(data_df.columns) > 0: #set master data frame
        dist_df = data_df.copy()
        first = False

    elif first == False and len(data_df.columns) > 0:
        print('merging dataframes')
        dist_df=pd.concat([dist_df, data_df]) #concat new data to master dataframe

    else:
        print('no data returned from query')

    path = '../linked_dist_stats/' #establish data file path
    if not os.path.exists(path):
        os.makedirs(path)
        
    print(dist_df.tail()) #check dataframe contens

#    dist_df['name'] = bin_df.name.loc[index] #set bin ranges as name
    dist_df.to_csv(path + 'distribution_edit_fails.csv') #save dataframe to file

SELECT '0 to 20' AS bin, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 7.0 AND 349.0) AND
    (income::INT BETWEEN 0 AND 20) AND property_type IN ('1') AND loan_type IN ('1') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
setting column names
       bin  count
0  0 to 20    661
SELECT '20 to 50' AS bin, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 8.0 AND 450.8199999999924) AND
    (income::INT BETWEEN 20 AND 50) AND property_type IN ('1') AND loan_type IN ('1') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
merging dataframes
       bin  count         0       1
0  0 to 20  661.0       NaN     NaN
0      NaN    NaN  20 to 50  1180.0
SELECT '50 to 80' AS bin, COUNT(amount) FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 14.0 AND 570.0) AND
    (income::INT BETWEEN 50 AND 80) AND pr

In [ ]:
#get rows of edit fails
#get edit fails based on distribution method
#this cell retrieves counts of edit fails by year (year must be adjusted manually)
#FIXME: add second query to get individual records that failed

#kwargs are used to set filters for market product
#this script should be tested with all loan types and property types
kwargs= {'table':'hmdalar2014',
         'prop_type':"'1'",
        'loan_purp':"'1'",
        'loan_type':"'1'",
        'occupancy':"'1'", 
        'lien':"'1'"}


first = True #set boolean flag to instantiate first instance of dataframe

for index, row in bin_df.iterrows(): #iterate over bins of income from bin_df 
#    print(index)
    #set variable arguments for sql query filters
    kwargs['name'] = bin_df.name.loc[index]
    kwargs['inc_min'] = int(kwargs['name'].rsplit(' ', 2)[0])
    kwargs['inc_max'] = int(kwargs['name'].rsplit(' ', 2)[2])
    kwargs['amt_min'] = bin_df.q02.loc[index]
    kwargs['amt_max'] = bin_df.q98.loc[index]
    
    sql = get_row_edit_fails(**kwargs) #format query using kwargs dictionary
    print(sql) #check query text
    cur.execute(sql,) #execute SQL
    data_df = pd.DataFrame(cur.fetchall()) #convert results to dataframe
    
    col_names = [desc[0] for desc in cur.description] # get column names from SQL table
    
    if len(data_df.columns) == len(col_names): #name dataframe columns if query not empty
        print('setting column names')
        data_df.columns = col_names

    if first == True and len(data_df.columns) > 0: #set master data frame
        dist_df = data_df.copy()
        first = False

    elif first == False and len(data_df.columns) > 0:
        print('merging dataframes')
        dist_df=pd.concat([dist_df, data_df]) #concat new data to master dataframe

    else:
        print('no data returned from query')

    path = '../linked_dist_stats/' #establish data file path
    if not os.path.exists(path):
        os.makedirs(path)
        
    print(dist_df.tail()) #check dataframe contens

    dist_df['name'] = bin_df.name.loc[index] #set bin ranges as name
    dist_df.to_csv(path + 'distribution_row_edit_fails.csv') #save dataframe to file

SELECT '0 to 20' AS bin, agency, RID, income, amount, sequence FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 349.0 AND 7.0) AND
    (income::INT BETWEEN 0 AND 20) AND property_type IN ('1') AND loan_type IN ('1') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND lien IN ('1') 
    AND amount NOT ILIKE '%NA%' AND income NOT ILIKE '%NA%' ;
setting column names
           bin agency         rid income amount sequence
33359  0 to 20      1  0000014740   0020  00036  0000625
33360  0 to 20      3  0000001821   0020  00014  0000040
33361  0 to 20      3  0000001821   0018  00024  0000049
33362  0 to 20      9  0000451965   0015  00020  0331686
33363  0 to 20      9  0000451965   0015  00030  0585171
SELECT '20 to 50' AS bin, agency, RID, income, amount, sequence FROM hmdalar2014 WHERE (amount::INT NOT BETWEEN 450.8199999999924 AND 8.0) AND
    (income::INT BETWEEN 20 AND 50) AND property_type IN ('1') AND loan_type IN ('1') AND 
    loan_purpose IN ('1') AND occupancy IN ('1') AND 

In [ ]:
#compress tail bins for counts df into one 600+ income
fail_counts_df = pd.read_csv(path + 'distribution_edit_fails.csv')